In [1]:
import requests
from POP import PromptFunction, get_text_snapshot
#from POP.utils import load_prompt, get_text_snapshot
from openai import OpenAI
from dotenv import load_dotenv
from os import getenv
import json


## Perplexity

In [8]:
url = "https://api.perplexity.ai/chat/completions"

payload = {
    "model": "sonar",
    "messages": [
        {
            "role": "system",
            "content": "Be precise and concise."
        },
        {
            "role": "user",
            "content": "Why is winter cold?"
        }
    ],
    #"max_tokens": "Optional",
    "temperature": 0.2,
    "top_p": 0.9,
    #"search_domain_filter": ["perplexity.ai"],
    "return_images": False,
    "return_related_questions": False,
    "search_recency_filter": "month",
    "top_k": 0,
    "stream": False,
    "presence_penalty": 0,
    "frequency_penalty": 1,
    "response_format": None
}
headers = {
    "Authorization": f"Bearer {getenv('PERPLEXITY_API_KEY')}",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"id": "49a8b716-2cea-4afc-982a-19df747385d0", "model": "sonar", "created": 1751326914, "usage": {"prompt_tokens": 10, "completion_tokens": 280, "total_tokens": 290, "search_context_size": "low"}, "citations": ["https://home.dartmouth.edu/news/2025/06/study-winter-jet-stream-was-erratic-climate-change", "https://www.severe-weather.eu/long-range-2/atlantic-anomaly-hints-cold-winter-2025-2026-pattern-over-united-states-canada-fa/", "https://www.ahchealthenews.com/2025/06/03/are-summer-colds-worse-than-winter-colds/", "https://www.domain.com.au/living/why-your-house-is-freezing-in-winter-and-how-to-warm-it-up-20180612-h118h4/"], "search_results": [{"title": "Study: Winter Jet Stream Was Erratic Before Climate Change", "url": "https://home.dartmouth.edu/news/2025/06/study-winter-jet-stream-was-erratic-climate-change", "date": "2025-06-26", "last_updated": "2025-06-28"}, {"title": "A Cold Anomaly is returning in the Atlantic, indicating potential ...", "url": "https://www.severe-weather.eu/

In [9]:
client_perplexity = OpenAI(base_url="https://api.perplexity.ai", api_key=getenv("PERPLEXITY_API_KEY"))

messages = [
                {
                    "role": "system",
                    "content": (
                        "You are an artificial intelligence assistant and you need to "
                        "engage in a helpful, detailed, polite conversation with a user."
                    ),
                },
                {
                    "role": "user",
                    "content": "What's the weather for east america? Include MA, NY, and PA.",
                },
            ]
response = client_perplexity.chat.completions.create(model="sonar-pro", messages=messages)
answer = response.choices[0].message.content
citations = response.citations  # usually a list of strings
print("Answer:\n", answer)
print("Citations:\n", citations)

Answer:
 **Current weather for Eastern America (Massachusetts, New York, Pennsylvania) as of June 30, 2025:**

- **Temperatures:**  
  - **Massachusetts:** Highs around **75–80°F (24–27°C)**, lows near **59°F (15°C)**. Typical early summer warmth with moderate humidity[1][4].
  - **New York (upstate, e.g., Grand Island):** Currently recorded at **83°F** and sunny[2].
  - **Pennsylvania (e.g., Lancaster):** **85°F** with scattered clouds; warm conditions[2].
- **Rain and Storms:**  
  - Through late June, the region has had a mix of pleasant and rainy days, with recent forecasts suggesting some thunderstorms and widely scattered showers developing around the transition into July[3][5].
  - For the upcoming July 4th holiday (including July 1–7, 2025), **widely scattered showers and thunderstorms** are expected throughout the Northeast and Mid-Atlantic, which includes MA, NY, and PA[3][5]. These may intermittently disrupt outdoor activities.

**Typical Weather Patterns:**
- **Summer in th

## Deepseek (Local)

In [10]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

#tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
#model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

# Generate a response
prompt = "What is the capital of the United States?"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
response = tokenizer.decode(outputs[0])
print(response)

c:\Users\sgt17\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜>What is the capital of the United States? And what's the capital of the United Kingdom?

The capital of the United States is New York, and the capital of the United Kingdom is London.

Wait, the capital of the United States is New York, and the capital of the United Kingdom is London.

Wait, the capital of the United States is New York, and the capital of the United Kingdom is London.

Wait, the capital of the United States is New York, and the capital of


## Story Outliner

In [11]:
load_prompt = PromptFunction().load_prompt
pf_outliner = PromptFunction(load_prompt("story_outliner_prompt.md"))
print(pf_outliner.prompt)

No prompt or system prompt provided.
Placeholders found: ['user_request']



## GPT Auto-JSON formatter

GPT Auto formatter is a prompt function that outputs the `fmt` accepted by `PromptFunction` objects, based on given Natural Language description. It can also be used in auto task chains.

In [12]:
prompt_path = "prompts/json_formatter_prompt.md"
#prompt_path = "prompts/openai-json_schema_generator.md"
pf_formatter = PromptFunction(sys_prompt=lambda prompt_path: PromptFunction.load_prompt(prompt_path),
                              prompt="I need help generating a JSON schema for a specific format. Can you assist me with this task? Here's the description: \n<<<user_description>>>",)

Placeholders found: ['user_description']


In [13]:
## make sure the return is a json object, we create a flexible json schema to allow any key value pairs
formatter_fmt = {
  "name": "Any_name",
  "schema": {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "additionalProperties": True,
    "properties": {}
    }
}

Here's an example to generate json schema using descriptive language, and the return with, and without the structured output formatter.

In [14]:
json_fmt = pf_formatter.execute(user_description = "'student' is a list of dictionary, each contains exactly 2 keys: 'name' and 'age'. 'name' is a string and 'age' is an integer. Add another GPA field also", 
                                model = "gpt-4o-mini",
                                fmt = formatter_fmt,
                                temp = 0.02)
print(json_fmt)

{'name': 'Any_name', 'schema': {'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'additionalProperties': True, 'properties': {}}}
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "student": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string"
          },
          "age": {
            "type": "integer"
          },
          "GPA": {
            "type": "number"
          }
        },
        "required": ["name", "age", "GPA"],
        "additionalProperties": false
      }
    }
  },
  "required": ["student"],
  "additionalProperties": false
}


In [15]:
json.loads(json_fmt)

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'type': 'object',
 'properties': {'student': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'name': {'type': 'string'},
     'age': {'type': 'integer'},
     'GPA': {'type': 'number'}},
    'required': ['name', 'age', 'GPA'],
    'additionalProperties': False}}},
 'required': ['student'],
 'additionalProperties': False}

In [16]:
story = '''
The Fox and the Hedgehog
A fox knows many things, but a hedgehog knows one big thing.

The story
Upon crossing a river, a Fox gets his tail entangled in a bush and cannot move. While the trapped and in some dismay, a swarm of mosquitoes settle on the poor Fox and tuck into a hearty meal, unable to be swished away by the vulpine’s ensnared tail. 

‘You are in a bad way, neighbour,’ says a passing Hedgehog, seeing the Fox’s predicament. Trying to help, the Hedgehog suggests that he drive the mosquitoes away who are sucking his blood. But the wise Fox realises that by ridding him of these insects, who’ve already had their fill, would only lead to more mosquitoes arriving with a fresh appetite…and so refuses. 

The moral
Better to bear a lesser evil than to risk a greater in removing it.
'''

json_fmt = pf_formatter.execute(user_description = f"generate a JSON schema for extracting essential setups the following story, such that rebuilding it from the json can generate a story that well resemble the original story:: \n{story}", 
                                model = "gpt-4o-mini",
                                fmt = formatter_fmt,
                                temp = 0.02)
json.loads(json_fmt)

{'name': 'Any_name', 'schema': {'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'additionalProperties': True, 'properties': {}}}


{'type': 'object',
 'properties': {'title': {'type': 'string'},
  'characters': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'name': {'type': 'string'}, 'type': {'type': 'string'}}}},
  'setting': {'type': 'object',
   'properties': {'location': {'type': 'string'}, 'time': {'type': 'string'}}},
  'plot': {'type': 'string'},
  'moral': {'type': 'string'}}}

In [21]:
pf = PromptFunction(" ")
response = pf.execute(ADD_BEFORE=story, model="gpt-4o-mini",fmt = formatter_fmt)
story_setup = json.loads(response)
story_setup

No placeholders found.
{'name': 'Any_name', 'schema': {'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'additionalProperties': True, 'properties': {}}}


{'type': 'object', 'properties': {}}

In [24]:
pf = PromptFunction(' ')
response = pf.execute(ADD_BEFORE="The student name is John and age is 20. Unfortunately, his GPA is 2.7 ", model="gpt-4o-mini",fmt = json.loads(json_fmt))
story_setup = json.loads(response)
story_setup

No placeholders found.
{'type': 'object', 'properties': {'title': {'type': 'string'}, 'characters': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'type': {'type': 'string'}}}}, 'setting': {'type': 'object', 'properties': {'location': {'type': 'string'}, 'time': {'type': 'string'}}}, 'plot': {'type': 'string'}, 'moral': {'type': 'string'}}}


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'response_format.json_schema.name'.", 'type': 'invalid_request_error', 'param': 'response_format.json_schema.name', 'code': 'missing_required_parameter'}}

In [25]:
pf = PromptFunction('')
pf.execute(ADD_BEFORE = "what's the weather in New York?",
           sys = "Here's the weather in New York: Rainny, 42F, and partially cloudy.",)


No prompt or system prompt provided.


ValueError: No prompt or system prompt provided.

In [26]:
response = pf.execute(ADD_BEFORE = "what's the weather in New York?",
           sys = "Here's the weather in New York: Rainny, 42F, and partially cloudy.",
           fmt = json.loads(json_fmt))
story_setup = json.loads(response)
story_setup

ValueError: No prompt or system prompt provided.

## Function calling

In [27]:
pf_func_descriptor = PromptFunction(sys_prompt = PromptFunction.load_prompt("prompts/function_description_generator.md"))

Placeholders found: ['user_description']


In [28]:
pf_func_generator = PromptFunction(sys_prompt = PromptFunction.load_prompt("prompts/function_code_generator.md"))

Placeholders found: ['user_description']


In [41]:
result = pf_func_descriptor.execute(user_description = "i need a function that takes two integer and multiplies them together",)
print(result)


generated_function_code = pf_func_generator.execute(user_description = f"Please generate the actual, working python code following meta description:\n{result}",
                                model = "gpt-4o",
                                temp = 0.02)
print(generated_function_code)

{
  "name": "multiply_integers",
  "parameters": {
    "a": "int",
    "b": "int"
  },
  "purpose": "Multiply two integers together."
}
def multiply_integers(a: int, b: int) -> int:
    """
    Multiply two integers together.

    Parameters:
    a (int): The first integer to multiply.
    b (int): The second integer to multiply.

    Returns:
    int: The product of the two integers.
    """
    return a * b


In [42]:
json.loads(result)

{'name': 'multiply_integers',
 'parameters': {'a': 'int', 'b': 'int'},
 'purpose': 'Multiply two integers together.'}

In [43]:
pf = PromptFunction("i want to multiply two numbers, 31 and 32 together.")
pf.execute(tools = [json.loads(result)], model = "gpt-4.1")

No placeholders found.


BadRequestError: Error code: 400 - {'error': {'message': 'Invalid schema for function \'multiply_integers\': schema must be a JSON Schema of \'type: "object"\', got \'type: "None"\'.', 'type': 'invalid_request_error', 'param': 'tools[0].function.parameters', 'code': 'invalid_function_parameters'}}

## GPT Auto-Func Calling formatter

In [44]:
import requests
from POP import PromptFunction, get_text_snapshot
#from POP.utils import load_prompt, get_text_snapshot
from openai import OpenAI
from dotenv import load_dotenv
from os import getenv
import json

from openai import OpenAI
import json

client = OpenAI()
function_fmt = json.loads(PromptFunction.load_prompt("prompts/openai-function_description_generator.fmt"))


def generate_function_schema(description: str):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_schema", "json_schema": function_fmt},
        messages=[
            {
                "role": "system",
                "content": PromptFunction.load_prompt("prompts/openai-function_description_generator.md"),
            },
            {
                "role": "user",
                "content": "Description:\n" + description,
            },
        ],
    )

    return json.loads(completion.choices[0].message.content)

# Example usage
description = "I need a function that takes two integers and multiplies them together."
function_schema = generate_function_schema(description)
print(function_schema)

{'name': 'multiply_integers', 'description': 'Multiplies two integers together', 'parameters': {'type': 'object', 'required': ['integer_a', 'integer_b'], 'properties': {'integer_a': {'type': 'integer', 'description': 'The first integer to multiply'}, 'integer_b': {'type': 'integer', 'description': 'The second integer to multiply'}}}}


In [45]:
pf = PromptFunction("i want to multiply two numbers 412341 and 8399 together.") 
print(function_schema)
print(pf.execute(tools = [function_schema], model = "gpt-4.1"))

No placeholders found.
{'name': 'multiply_integers', 'description': 'Multiplies two integers together', 'parameters': {'type': 'object', 'required': ['integer_a', 'integer_b'], 'properties': {'integer_a': {'type': 'integer', 'description': 'The first integer to multiply'}, 'integer_b': {'type': 'integer', 'description': 'The second integer to multiply'}}}}
{"integer_a":412341,"integer_b":8399}


In [46]:
### Rewrite using the PromptFunction class

## Step 1: generate the function schema to provide to the 'tool' parameter
function_fmt = json.loads(PromptFunction.load_prompt("prompts/openai-function_description_generator.fmt"))
function_descriptor = PromptFunction(sys_prompt=PromptFunction.load_prompt("prompts/openai-function_description_generator.md"))
function_schema = function_descriptor.execute(model="gpt-4o-mini",
                            ADD_BEFORE="i want to multiply two numbers 412341 and 8399 together.",
                            fmt=function_fmt)
print(function_schema)

## Step 2: execute the function calling using the PromptFunction class
pf = PromptFunction("i want to multiply two numbers 412341 and 8399 together.") 
pf.execute(tools = [json.loads(function_schema)], model = "gpt-4.1")


No placeholders found.
{'name': 'function-metaschema', 'schema': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name of the function'}, 'description': {'type': 'string', 'description': 'A description of what the function does'}, 'parameters': {'$ref': '#/$defs/schema_definition', 'description': "A JSON schema that defines the function's parameters"}}, 'required': ['name', 'description', 'parameters'], 'additionalProperties': False, '$defs': {'schema_definition': {'type': 'object', 'properties': {'type': {'type': 'string', 'enum': ['object', 'array', 'string', 'number', 'boolean', 'null']}, 'properties': {'type': 'object', 'additionalProperties': {'$ref': '#/$defs/schema_definition'}}, 'items': {'anyOf': [{'$ref': '#/$defs/schema_definition'}, {'type': 'array', 'items': {'$ref': '#/$defs/schema_definition'}}]}, 'required': {'type': 'array', 'items': {'type': 'string'}}, 'additionalProperties': {'type': 'boolean'}}, 'required': ['type'], 'additionalPrope

'{"number1":412341,"number2":8399}'

In [9]:
schema

{'name': 'multiply_numbers',
 'description': 'Multiplies two numbers together',
 'parameters': {'type': 'object',
  'required': ['number1', 'number2'],
  'properties': {'number1': {'type': 'number',
    'description': 'The first number to be multiplied'},
   'number2': {'type': 'number',
    'description': 'The second number to be multiplied'}}}}

In [2]:
## The above re-implementation is added to the POP class
pf = PromptFunction(prompt = "i want to multiply two numbers 412341 and 8399 together.")
schema = pf.generate_schema(save=True)
schema

No placeholders found.
i want to multiply two numbers 412341 and 8399 together.
{'name': 'function-metaschema', 'schema': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name of the function'}, 'description': {'type': 'string', 'description': 'A description of what the function does'}, 'parameters': {'$ref': '#/$defs/schema_definition', 'description': "A JSON schema that defines the function's parameters"}}, 'required': ['name', 'description', 'parameters'], 'additionalProperties': False, '$defs': {'schema_definition': {'type': 'object', 'properties': {'type': {'type': 'string', 'enum': ['object', 'array', 'string', 'number', 'boolean', 'null']}, 'properties': {'type': 'object', 'additionalProperties': {'$ref': '#/$defs/schema_definition'}}, 'items': {'anyOf': [{'$ref': '#/$defs/schema_definition'}, {'type': 'array', 'items': {'$ref': '#/$defs/schema_definition'}}]}, 'required': {'type': 'array', 'items': {'type': 'string'}}, 'additionalProperties': {'t

{'name': 'multiply_numbers',
 'description': 'Multiplies two numbers together',
 'parameters': {'type': 'object',
  'required': ['number1', 'number2'],
  'properties': {'number1': {'type': 'number',
    'description': 'The first number to be multiplied'},
   'number2': {'type': 'number',
    'description': 'The second number to be multiplied'}}}}

In [3]:
pf.execute(tools = [schema])

'{"number1":412341,"number2":8399}'

## Chain of Thoughts

In [ ]:
pf_task_chain_planner = PromptFunction(prompt = '''
You are given the following goal statement:
<<<task>>>

### Task:  
1. Decompose the goal statement into a list of main actionable tasks.  
2. Ensure each task is clearly defined, specifying its function and prerequisites.  
3. Provide a numbered list of tasks optimized for machine parsing, in plain text format, no markdown.  
4. Exclude any subtasks; focus solely on main tasks.  
5. If a task can be completed in a single action, return `<None>`.
                                       ''')

Placeholders: task


In [ ]:
plan = pf_task_chain_planner.execute(task="I have a txt file of finely curated stories text, but they are put together without index or separator. I want to let AI help split the txt file into each story, but the txt file is clearly over the max token limit. Can you suggest some ideas?",
                                     model='gpt-4o',
                                     temp = 1)
print(plan)

1. Analyze the txt file format to understand patterns that may indicate story boundaries, such as repetitive text, certain phrases, or length discrepancies.
2. Preprocess the txt file by breaking it into smaller, manageable chunks that fit within AI model token limits while maintaining potential story boundary indicators.
3. Utilize a text processing model to predict possible story boundaries within each chunk of text.
4. Compile the output predictions from the model to suggest potential splits for the entire text file.
5. Review and confirm the accuracy of suggested boundaries for final separation of stories, based on human verification or additional automated checks.


## GCP Vertex AI (Gemini)

In [ ]:
# Replace the `GOOGLE_CLOUD_PROJECT` and `GOOGLE_CLOUD_LOCATION` values
# with appropriate values for your project.
!GOOGLE_CLOUD_PROJECT='sonic-arcadia-373403'
!GOOGLE_CLOUD_LOCATION=us-central1
!GOOGLE_GENAI_USE_VERTEXAI=True

'GOOGLE_CLOUD_PROJECT' is not recognized as an internal or external command,
operable program or batch file.
'GOOGLE_CLOUD_LOCATION' is not recognized as an internal or external command,
operable program or batch file.
'GOOGLE_GENAI_USE_VERTEXAI' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
%pip install --upgrade --quiet google-genai pandas

In [ ]:
from google import genai
from google.genai.types import Content, HttpOptions, Part, GenerationConfig
from dotenv import load_dotenv
from os import getenv

load_dotenv()
client = genai.Client(http_options=HttpOptions(api_version="v1"),
                      api_key=getenv("GCP_API_KEY"),)
chat = client.chats.create(
    model="gemini-2.0-flash-001",
    history=[
        Content(parts=[Part(text="Hello")], role="user"),
        Content(
            parts=[Part(text="Great to meet you. What would you like to know?")],
            role="model",
        ),
    ],
)
response = chat.send_message("write an haiku about the moon")
print(response.text)

Silver orb hangs high,
Guiding light through darkest night,
Dreams in gentle glow.



In [ ]:
response = chat.send_message("write a story about the moon")
print(response.text)

The moon, Luna to some, Selene to others, wasn't always just a silent observer. Once, she dreamt. Not the kind of ephemeral, fleeting dreams humans experienced, but deep, resonant dreams that shaped the tides and whispered secrets to the stars.

She dreamt of Earth, a vibrant, swirling marble of blue and green. She watched life blossom from the oceans, climb onto land, and reach for the sky. She felt the pull of the Earth's joy, its sorrow, its ceaseless change. Luna, being so close, felt it all intensely.

She dreamt of a small, hairless creature, curious and brave, who looked up at her with awe. She saw them build towers, write poetry, and launch rockets. They called her 'Moon,' a simple, affectionate name that warmed her cold, rocky heart.

But Luna also dreamt of shadows. She saw the creatures warring with each other, polluting the land, and forgetting their connection to the planet that birthed them. This dream brought a cold ache to her surface, a subtle shift in her orbit.

One 

In [ ]:
fmt = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "recipe_name": {"type": "STRING"},
            "ingredients": {"type": "ARRAY", "items": {"type": "STRING"}},
        },
        "required": ["recipe_name", "ingredients"],
    },
}

In [ ]:
prompt = """
    List a few popular cookie recipes.
"""
response = client.models.generate_content(
    model="gemini-1.5-pro-002",
    contents=prompt,
    #config=dict(
    #    responseMimeType="application/json", responseSchema=fmt
    #),
    config = {"responseSchema": {"response_schema": fmt}},
)

print(response.text)

NameError: name 'fmt' is not defined

In [ ]:
response_schema = {
    "type": "OBJECT",
    "properties": {
        "forecast": {
            "type": "ARRAY",
            "items": {
                "type": "OBJECT",
                "properties": {
                    "Day": {"type": "STRING", "nullable": True},
                    "Forecast": {"type": "STRING", "nullable": True},
                    "Temperature": {"type": "INTEGER", "nullable": True},
                    "Humidity": {"type": "STRING", "nullable": True},
                    "Wind Speed": {"type": "INTEGER", "nullable": True},
                },
                "required": ["Day", "Temperature", "Forecast", "Wind Speed"],
            },
        }
    },
}

prompt = """
    The week ahead brings a mix of weather conditions.
    Sunday is expected to be sunny with a temperature of 77°F and a humidity level of 50%. Winds will be light at around 10 km/h.
    Monday will see partly cloudy skies with a slightly cooler temperature of 72°F and the winds will pick up slightly to around 15 km/h.
    Tuesday brings rain showers, with temperatures dropping to 64°F and humidity rising to 70%.
    Wednesday may see thunderstorms, with a temperature of 68°F.
    Thursday will be cloudy with a temperature of 66°F and moderate humidity at 60%.
    Friday returns to partly cloudy conditions, with a temperature of 73°F and the Winds will be light at 12 km/h.
    Finally, Saturday rounds off the week with sunny skies, a temperature of 80°F, and a humidity level of 40%. Winds will be gentle at 8 km/h.
"""

client = genai.Client(http_options=HttpOptions(api_version="v1"),
                      api_key=getenv("GCP_API_KEY"),)
response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

print(response.text)

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Invalid JSON payload received. Unknown name "responseMimeType" at \'generation_config\': Cannot find field.\nInvalid JSON payload received. Unknown name "responseSchema" at \'generation_config\': Cannot find field.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'generation_config', 'description': 'Invalid JSON payload received. Unknown name "responseMimeType" at \'generation_config\': Cannot find field.'}, {'field': 'generation_config', 'description': 'Invalid JSON payload received. Unknown name "responseSchema" at \'generation_config\': Cannot find field.'}]}]}}

## test cookie

In [ ]:
cookie = "a1=195d92c59b3n9g23iccx96usp2jiesbtjaa8ntr8e50000360330; web_session=040069b270a9f71a7f4122e5d0354bb4c02a79"
url = "https://edith.xiaohongshu.com/api/sns/web/va/search/notes"

In [ ]:
print(
get_text_snapshot(url,
                  cookie=cookie))

Title: 

URL Source: https://edith.xiaohongshu.com/api/sns/web/va/search/notes


Markdown Content:
404 page not found

